In [134]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from unidecode import unidecode
from scipy.stats import norm

In [3]:
def fix_column_names(df):
    dict_fix = {'.':' ','-':' ','/':' ',' ':'_'}
    column_names = df.columns
    new_column_names = []
    for col in column_names:
        # Remove acentos e substitui caracteres especiais
        fixed_col = unidecode(col)
        for char in dict_fix:
            fixed_col = fixed_col.replace(char, dict_fix[char])
        new_column_names.append(fixed_col)
    return new_column_names

In [119]:
# importing data
df_base_article = pd.read_excel("BaseArticle.xlsx")
df_base_prev = pd.read_excel('Base ventes et prévisions.xlsx')
df_data_modele = pd.read_excel('Data Modèle.xlsx', sheet_name = 'Stocks')
df_stock_central = pd.read_excel('Stock Central.xlsx')
df_stock_local = pd.read_excel('Stock Local.xlsx', header=1)

In [120]:
# cleaning column names
df_base_article.columns = fix_column_names(df_base_article)
df_base_article = df_base_article.rename(columns={'Classe_':'Classe'})

df_base_prev.columns = fix_column_names(df_base_prev)

df_data_modele.columns = fix_column_names(df_data_modele)

df_stock_central.columns = fix_column_names(df_stock_central)

df_stock_local.columns = fix_column_names(df_stock_local)

In [94]:
# making that every negative number on Historique vente is 0
df_base_prev.loc[df_base_prev.query('Historique_vente<0').index,'Historique_vente'] = 0

In [109]:
# calculating the mean value of forecast respecting the product and the entrepot
mean_forecast = df_base_prev.groupby(['Product','Subsidiary'])[['Forecast_M_3']].mean().reset_index()
df_merge_forecast = pd.merge(mean_forecast, df_base_prev, how='left')

# joining dataframes
df_merge_forecast = pd.merge(mean_forecast, df_base_prev, how='left', on=['Product','Subsidiary'])

# attributing the mean value calculated to nan
condition = df_merge_forecast['Forecast_M_3_y'].isna() & ~df_merge_forecast['Forecast_M_3_x'].isna()
df_merge_forecast.loc[condition, 'Forecast_M_3_y'] = df_merge_forecast.loc[condition, 'Forecast_M_3_x']

# renaming columns
df_merge_forecast = df_merge_forecast.drop('Forecast_M_3_x', axis=1).rename(columns={'Forecast_M_3_y':'Forecast'})

# calculating the standard deviation, in respect to the product and the entrepot
df_merge_forecast['Difference'] = df_merge_forecast['Historique_vente'] - df_merge_forecast['Forecast']
df_std = df_merge_forecast.groupby(['Product','Subsidiary'])[['Difference']].std().reset_index().drop('Difference', axis=1)
df_merge_forecast = pd.merge(df_merge_forecast, df_std, on=['Subsidiary','Product']).rename(columns={'Difference':'Std_deviation'})

# changing names
list_entrepots = {'France':'FR','Belgium':'BE','Spain':'ES', 'Germany':'DE', 'Portugal':'PT', 'Italy':'IT', 'UK':'UK', 'Swiss':'CH', 'Poland':'PL'}
df_merge_forecast['Subsidiary'] = df_merge_forecast['Subsidiary'].replace(list_entrepots)
df_merge_forecast = df_merge_forecast.rename(columns={'Subsidiary':'Filiale','Product_1':'Description'})    

In [145]:
df_qty = pd.merge(df_merge_forecast, df_stock_local, on=['Filiale','Mois','Description','Range','Category'], how='outer').drop_duplicates(['Filiale','Mois','Description'])

In [148]:
df_qty_clean = df_qty.loc[~df_qty['Stock_fin_de_mois_(QTY)'].isna()&~df_qty['Std_deviation'].isna()]
df_qty_clean['Stock_sec'] = df_qty_clean['Stock_fin_de_mois_(QTY)'] - (df_qty_clean['Forecast'] / 2)

delais = np.sqrt(1/52+1/12)

# Calcul du K de la formule
df_qty_clean['K'] = df_qty_clean['Stock_sec']/(df_qty_clean['Std_deviation']*delais)

# Calcul du taux de service par la 'cumulative distribution function'
df_qty_clean['Taux_de_service'] = norm.cdf(df_qty_clean['K'])

# Formatage de l'affichage du taux de service sous la forme d'un pourcentage à 4 chiffres
df_qty_clean['Taux_de_service'] = df_qty_clean['Taux_de_service'].apply(lambda x: '{:.2f}%'.format(x * 100))

C:\Users\10\AppData\Local\Temp\ipykernel_4808\4072659158.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_qty_clean['Stock_sec'] = df_qty_clean['Stock_fin_de_mois_(QTY)'] - (df_qty_clean['Forecast'] / 2)
C:\Users\10\AppData\Local\Temp\ipykernel_4808\4072659158.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_qty_clean['K'] = df_qty_clean['Stock_sec']/(df_qty_clean['Std_deviation']*delais)
C:\Users\10\AppData\Local\Temp\ipykernel_4808\4072659158.py:10: SettingWithCopyWarning: 
A value is trying t

In [149]:
df_qty_clean

,Product,Filiale,Mois,Brand,Range,Description,Category,Historique_vente,Forecast,Std_deviation,Code_article,Marque,Classe_ABC_filiale,Stock_fin_de_mois_(QTY),Stock_sec,K,Taux_de_service
24,1000001.0,DE,2018-09-01,MARQUE B,Parfum,PRODUIT 1000001,Produit vente,1307.02,3360.908571,-2053.888571,1000001.0,MARQUE B,A,394.0,-1286.454286,1.955779,97.48%
25,1000001.0,DE,2018-04-01,MARQUE B,Parfum,PRODUIT 1000001,Produit vente,12187.53,3323.871818,8863.658182,1000001.0,MARQUE B,A,1354.0,-307.935909,-0.108480,45.68%
26,1000001.0,DE,2018-08-01,MARQUE B,Parfum,PRODUIT 1000001,Produit vente,159.00,216.000000,-57.000000,1000001.0,MARQUE B,A,493.0,385.000000,-21.090563,0.00%
27,1000001.0,DE,2018-12-01,MARQUE B,Parfum,PRODUIT 1000001,Produit vente,1954.47,1740.108774,214.361226,1000001.0,MARQUE B,A,294.0,-576.054387,-8.391113,0.00%
28,1000001.0,DE,2018-07-01,MARQUE B,Parfum,PRODUIT 1000001,Produit vente,2419.99,3372.373161,-952.383161,1000001.0,MARQUE B,A,654.0,-1032.186581,3.384144,99.96%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26063,5000018.0,IT,2018-12-01,MARQUE B,Divers,PRODUIT 5000018,Produit vente,12.48,0.000000,12.480000,5000018.0,MARQUE B,C,24.0,24.000000,6.004806,100.00%
26064,5000018.0,IT,2018-11-01,MARQUE B,Divers,PRODUIT 5000018,Produit vente,24.96,0.000000,24.960000,5000018.0,MARQUE B,C,25.0,25.000000,3.127503,99.91%
26065,5000018.0,IT,2018-01-01,MARQUE B,Divers,PRODUIT 5000018,Produit vente,5.00,0.000000,5.000000,5000018.0,MARQUE B,C,0.0,0.000000,0.000000,50.00%
26066,5000018.0,IT,2018-02-01,MARQUE B,Divers,PRODUIT 5000018,Produit vente,4.00,0.000000,4.000000,5000018.0,MARQUE B,C,0.0,0.000000,0.000000,50.00%
